In [1]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn

from mindnlp import load_dataset

from mindnlp.engine import Trainer, Evaluator
from mindnlp.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp.metrics import Accuracy

mindspore.set_context(device_target="Ascend")

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 数据预处理

In [2]:
# 加载数据集
glue_ds = load_dataset('glue','qnli', split=['train', 'test'])
glue_train = glue_ds['train']
glue_test = glue_ds['test']

In [3]:
glue_train.get_dataset_size()

104743

In [4]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=512, batch_size=16, shuffle=False):
    def tokenize(text):
        tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized[0], tokenized[1]

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="sentence", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})

    return dataset

In [5]:
from mindnlp.transformers import GPTTokenizer
# tokenizer
gpt_tokenizer = GPTTokenizer.from_pretrained('openai-gpt', from_pt=True)

# add sepcial token: <PAD>
special_tokens_dict = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
}
num_added_toks = gpt_tokenizer.add_special_tokens(special_tokens_dict)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [6]:
# split train dataset into train and valid datasets
glue_train, glue_val = glue_train.split([0.7, 0.3])

In [7]:
dataset_train = process_dataset(glue_train, gpt_tokenizer, shuffle=True)
dataset_val = process_dataset(glue_val, gpt_tokenizer)
dataset_test = process_dataset(glue_test, gpt_tokenizer)

In [8]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[16, 70], dtype=Int64, value=
 [[  481,  7585, 30317 ... 40480, 40480, 40480],
  [ 4876,   260, 18890 ... 40480, 40480, 40480],
  [10431,   839, 22178 ... 40480, 40480, 40480],
  ...
  [  481,  1529,   498 ... 40480, 40480, 40480],
  [  481, 10638,   498 ... 40480, 40480, 40480],
  [ 6771,   240,  4695 ... 40480, 40480, 40480]]),
 Tensor(shape=[16, 70], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[16], dtype=String, value= ['What is two islands that are part of the British Isles?',
  'Where was the basic power in Rome to be found?',
  'Along with gardening, animal husbandry and child-care, what work did medieval peasant women do?',
  'How long is the CAP?',
  'Which game publication does Jeff Gerstmann work for?',
  'Bach was influenced by a popular A cappella composer heavily for what composition?',
  'Is maple wood diffuse-p

# 模型训练

In [9]:
from mindnlp.transformers import GPTForSequenceClassification

# set bert config and define parameters for training
model = GPTForSequenceClassification.from_pretrained('openai-gpt', from_pt=True, num_labels=2)
model.config.pad_token_id = gpt_tokenizer.pad_token_id
model.resize_token_embeddings(model.config.vocab_size + 3)

optimizer = nn.Adam(model.trainable_params(), learning_rate=2e-5)

metric = Accuracy()

# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='gpt_glue_finetune', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='gpt_glue_finetune_best', auto_load=True)

trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_train, metrics=metric,
                  epochs=3, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=False)

The following parameters in models are missing parameter:
['score.weight']
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/ipykernel/iostream.py:153: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0xfffe2ea4e340>
  self._event_pipes[threading.current_thread()] = event_pipe


In [ ]:
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0:   0%|          | 0/4583 [00:00<?, ?it/s]

# 模型评估

In [ ]:
evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
evaluator.run(tgt_columns="labels")